In [ ]:
import os
import sqlite3
import pandas as pd
from datetime import datetime

ROOT_DIR = "/home/admin/Desktop/field_data_management_platform"
DB_PATH = os.path.join(ROOT_DIR, "data", "field_data.sqlite")

query_date = datetime(2024, 1, 1)
start_ts = query_date.strftime("%Y-%m-%d 00:00:00")
end_ts = query_date.strftime("%Y-%m-%d 23:59:59")

with sqlite3.connect(DB_PATH) as conn:
    sp_inv30 = pd.read_sql_query(
        """
        SELECT *
        FROM processed_sp
        WHERE inverterNo = ? AND time BETWEEN ? AND ?
        ORDER BY time
        """,
        conn,
        params=["30", start_ts, end_ts],
    )

sp_inv30.head()

## Load Libraries

In [1]:
# https://medium.com/@nrk25693/how-to-add-your-conda-environment-to-your-jupyter-notebook-in-just-4-steps-abeab8b8d084
! pip3 install paramiko 
! pip3 install bs4 
! pip3 install requests 
! pip3 install numpy
! pip3 install pandas
! pip3 install selenium
! pip3 install sqlalchemy
! pip3 install mysql-connector-python
! python -m pip install mysql-connector-python
! pip3 install --upgrade --force-reinstall cryptography

  Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl.metadata (5.7 kB)
  Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.6 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
Using cached cryptography-46.0.3-cp311-abi3-manylinux_2_34_x86_64.whl (4.5 MB)
Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (219 kB)
Using cached pycparser-2.23-py3-none-any.whl (118 kB)
  Attempting uninstall: pycparser
    Found existing installation: pycparser 2.23
    Uninstalling pycparser-2.23:
      Successfully uninstalled pycparser-2.23
  Attempting uninstall: cffi
    Found existing installation: cffi 2.0.0
    Uninstalling cffi-2.0.0:
      Successfully uninstalled cffi-2.0.0━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [cffi]
  Attempting uninstall: cryptography━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [cffi]
    Found existing installation: cryptography 46.0.3━━━━━━━━━━ 1/3 [cffi]
    Uninstalling cryptogr

In [2]:
import numpy as np
from datetime import datetime, timedelta
import pandas as pd
import os
import sys
import time

from sp import SunnyPortal as sp

from ae import AlsoEnergy as ae

from ws import WeatherStation as ws

from de import DominionEnergySFTP as de_sftp

from mysql_lib import mySQLConnect as mysql_lib

from config import (
    AE_USERNAME, AE_PASSWORD,
    SP_USERNAME, SP_PASSWORD,
    WS_API_KEY,
    MYSQL_HOST, MYSQL_USERNAME, MYSQL_PASSWORD, MYSQL_DATABASE,
    SFTP_HOST, SFTP_USERNAME, SFTP_PRIVATE_KEY_PATH
)

## Path

In [16]:
# path - using os.path.join for better cross-platform compatibility
root_path = os.getcwd()
sp_path = os.path.join(root_path, "sp", "sp_data/")
ae_path = os.path.join(root_path, "ae", "ae_data/")
de_path = os.path.join(root_path, "de", "de_data/")
ws_path = os.path.join(root_path, "ws", "ws_data/")
tmy_path = os.path.join(root_path, "tmy/")

#driverPath = os.path.join(root_path, "chromedriver-linux64", "chromedriver")
#chromePath = os.path.join(root_path, "chrome-linux64", "chrome")
driverPath = os.path.join(root_path, "chromedriver_palmetto", "chromedriver")
chromePath = os.path.join(root_path, "chrome_palmetto", "chrome")

sys.path.append(driverPath)

In [4]:
sp_path, ae_path, ws_path

('/home/admin/Desktop/data_management_platform/sp/sp_data/',
 '/home/admin/Desktop/data_management_platform/ae/ae_data/',
 '/home/admin/Desktop/data_management_platform/ws/ws_data/')

## Date Range 

In [5]:
start_date = datetime(2024,1,1)
end_date = datetime(2025,12,31)
#end_date =  datetime.today() - timedelta(days=1) #datetime(2024,4,15)

## Other Parameters

In [6]:
interval = 5
thr = 0.4
sp_interval = 5
ae_interval = 1
tmy_interval = 60
sp_itv = int(tmy_interval/sp_interval)
ae_itv = int(tmy_interval/ae_interval)
num_display = 8
num_device = 3

## Acquire Data

In [7]:
def ae_download():
    """Download data from AlsoEnergy."""
    ae_object = ae.AlsoEnergy(ae_path, driverPath, chromePath)
    ae_object.setUserName(AE_USERNAME)
    ae_object.setPassword(AE_PASSWORD)
    ae_object.setStartDate(start_date)
    ae_object.setEndDate(end_date)
    ae_object.AlsoEnergy()

In [8]:
def sp_download():
    """Download data from SunnyPortal."""
    sp_object = sp.SunnyPortal(sp_path, driverPath, chromePath)
    sp_object.setUserName(SP_USERNAME)
    sp_object.setPassword(SP_PASSWORD)
    sp_object.setStartDate(start_date)
    sp_object.setEndDate(end_date)
    sp_object.SunnyPortal()

In [9]:
def ws_download():
    """Download data from Weather Station."""
    ws_object = ws.WeatherStation(ws_path, WS_API_KEY)
    ws_object.setStartDate(start_date)
    ws_object.setEndDate(end_date)
    ws_object.WeatherStation()

In [10]:
def de_download():
    """Download data from Dominion Energy SFTP."""
    sftp_private_key_path = os.path.join(root_path, SFTP_PRIVATE_KEY_PATH)
    dm_sftp_obj = de_sftp.DominionEnergySFTP(SFTP_HOST, SFTP_USERNAME, sftp_private_key_path)
    dm_sftp_obj.connect()
    
    # download file
    out_folder_name = "./outbound/"
    download_folder = de_path
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)
    
    out_file_list = dm_sftp_obj.read_file_list(out_folder_name)
    existing_files = os.listdir(download_folder) if os.path.exists(download_folder) else []
    
    for file in out_file_list:
        if file not in existing_files:
            dm_sftp_obj.download(file, download_folder)
        else:
            print(file, "existed!")

In [17]:
# Data Collection
sp_download()
#ae_download()
#ws_download()
#de_download()

Using Chrome binary: /home/admin/Desktop/data_management_platform/chromedriver_palmetto/chromedriver
Using ChromeDriver: /home/admin/Desktop/data_management_platform/chrome_palmetto/chrome
Error initializing ChromeDriver: Message: Service /home/admin/Desktop/data_management_platform/chrome_palmetto/chrome unexpectedly exited. Status code was: -5

Login Error:  Message: Service /home/admin/Desktop/data_management_platform/chrome_palmetto/chrome unexpectedly exited. Status code was: -5

sp_2024-01-01  existed!
sp_2024-01-02  existed!
sp_2024-01-03  existed!
sp_2024-01-04  existed!
sp_2024-01-05  existed!
sp_2024-01-06  existed!
sp_2024-01-07  existed!
sp_2024-01-08  existed!
sp_2024-01-09  existed!
sp_2024-01-10  existed!
sp_2024-01-11  existed!
sp_2024-01-12  existed!
sp_2024-01-13  existed!
sp_2024-01-14  existed!
sp_2024-01-15  existed!
sp_2024-01-16  existed!
sp_2024-01-17  existed!
sp_2024-01-18  existed!
sp_2024-01-19  existed!
sp_2024-01-20  existed!
sp_2024-01-21  existed!
sp_202

KeyboardInterrupt: 

## Load Data

### Load TMY as Reference

#### Load TMY Data

In [ ]:
tmy_list = os.listdir(tmy_path) if os.path.exists(tmy_path) else []
tmy_dfs = []
for file in tmy_list:
    try:
        if file.endswith(".csv"):
            file_path = os.path.join(tmy_path, file)
            tmy_df = pd.read_csv(file_path, delimiter=',', skiprows=2)
            tmy_dfs.append(tmy_df)
    except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError) as e:
        print(f"{file} data missed: {e}")
    except Exception as e:
        print(f"Unexpected error loading {file}: {e}")

if tmy_dfs:
    tmy_df = pd.concat(tmy_dfs, ignore_index=True)
else:
    raise ValueError("No TMY data files were successfully loaded. Please check the tmy_path directory.")

#### Calculate History High and Low

In [ ]:
# History High and Low
max_ghi = max(tmy_df['GHI'])
min_ghi = min(tmy_df['GHI'])
max_temp = max(tmy_df['Temperature'])
min_temp = min(tmy_df['Temperature'])
max_rh = max(tmy_df['Relative Humidity'])
min_rh = min(tmy_df['Relative Humidity'])
max_ghi, min_ghi, max_temp, min_temp, max_rh, min_rh

### Load Field Data (Sunny Portal, Also Energy, Weather Station)

#### Caluculate the Index

In [ ]:
total_days = (end_date-start_date).days + 1

sp_num_record = np.zeros((total_days),dtype=np.int64)
ae_num_record = np.zeros((total_days),dtype=np.int64)
ws_num_record = np.zeros((total_days),dtype=np.int64)

sp_start_index = np.zeros((total_days),dtype=np.int64)
sp_end_index = np.zeros((total_days),dtype=np.int64)

ae_start_index = np.zeros((total_days),dtype=np.int64)
ae_end_index = np.zeros((total_days),dtype=np.int64)

ws_start_index = np.zeros((total_days),dtype=np.int64)
ws_end_index = np.zeros((total_days),dtype=np.int64)


for i in range(total_days):
    current_date = start_date + timedelta(days=i)
    sp_time_list = pd.date_range(start=current_date.replace(hour=0, minute=0), end=current_date.replace(hour=23, minute=55), freq='5min', tz='US/Eastern')
    ae_time_list = pd.date_range(start=current_date.replace(hour=0, minute=0), end=current_date.replace(hour=23, minute=59), freq='1min', tz='US/Eastern')
    ws_time_list = pd.date_range(start=current_date.replace(hour=0, minute=30), end=current_date.replace(hour=23, minute=59), freq='60min', tz='US/Eastern')
    
    sp_num_record[i] = len(sp_time_list)
    ae_num_record[i] = len(ae_time_list)
    ws_num_record[i] = len(ws_time_list) 

    if(i==0):
        sp_start_index[i] = 0
        ae_start_index[i] = 0
        ws_start_index[i] = 0
        sp_end_index[i] = num_device*sp_num_record[i]
        ae_end_index[i] = ae_num_record[i]
        ws_end_index[i] = ws_num_record[i]
    else:
        sp_start_index[i] = sp_end_index[i-1]
        ae_start_index[i] = ae_end_index[i-1]
        ws_start_index[i] = ws_end_index[i-1]
        sp_end_index[i] = sp_start_index[i] + num_device*sp_num_record[i]
        ae_end_index[i] = ae_start_index[i] + ae_num_record[i]
        ws_end_index[i] = ws_start_index[i] + ws_num_record[i]

#### Load Data File: Also Energy(ae_df), SunnyPortal(sp_env_df, sp_op_df), Weather Station(ws_df)

In [ ]:
current_date = start_date
ae_dfs = []
sp_env_dfs = []
sp_op_dfs = []
ws_dfs = []
ws_tmy_dfs = []

missing_files = []

for i in range(total_days):
    current_date = start_date + timedelta(days=i)
    date_str = current_date.strftime("%Y-%m-%d")
    
    ae_filename = os.path.join(ae_path, f"ae_{date_str}.csv")
    sp_env_filename = os.path.join(sp_path, "environmental", f"sp_{date_str}.csv")
    sp_op_filename = os.path.join(sp_path, "operating", f"sp_{date_str}.csv")
    ws_filename = os.path.join(ws_path, f"ws_{date_str}.csv")
    
    try:
        if os.path.exists(ae_filename):
            ae_df = pd.read_csv(ae_filename, delimiter=',')
            ae_dfs.append(ae_df)
        else:
            missing_files.append(f"AE: {ae_filename}")
            
        if os.path.exists(sp_env_filename):
            sp_env_df = pd.read_csv(sp_env_filename, delimiter=',')
            sp_env_dfs.append(sp_env_df)
        else:
            missing_files.append(f"SP_ENV: {sp_env_filename}")
            
        if os.path.exists(sp_op_filename):
            sp_op_df = pd.read_csv(sp_op_filename, delimiter=',')
            sp_op_dfs.append(sp_op_df)
        else:
            missing_files.append(f"SP_OP: {sp_op_filename}")
            
        if os.path.exists(ws_filename):
            ws_df = pd.read_csv(ws_filename, delimiter=',')
            ws_dfs.append(ws_df)
        else:
            missing_files.append(f"WS: {ws_filename}")
            
    except Exception as e:
        print(f"Error loading data for {current_date}: {e}")
        missing_files.append(f"Error on {date_str}: {e}")

if missing_files:
    print(f"Warning: {len(missing_files)} files missing or errors encountered")
    print("First 10 missing files:", missing_files[:10])

# Concatenate list
if ae_dfs:
    ae_df = pd.concat(ae_dfs, ignore_index=True)
    # Delete the index column if it exists
    if ae_df.columns[0] == 'Unnamed: 0' or 'index' in ae_df.columns[0].lower():
        ae_df = ae_df.drop(ae_df.columns[0], axis=1)
else:
    raise ValueError("No AlsoEnergy data files were successfully loaded.")

if sp_env_dfs:
    sp_env_df = pd.concat(sp_env_dfs, ignore_index=True)
else:
    raise ValueError("No SunnyPortal environmental data files were successfully loaded.")

if sp_op_dfs:
    sp_op_df = pd.concat(sp_op_dfs, ignore_index=True)
else:
    raise ValueError("No SunnyPortal operating data files were successfully loaded.")

if ws_dfs:
    ws_df = pd.concat(ws_dfs, ignore_index=True)
else:
    raise ValueError("No Weather Station data files were successfully loaded.")

#### Clean Data : Ir is 0 except for the operating time

In [ ]:
mask_op = sp_op_df['ac_power'] == -1
mask_env = sp_env_df['ir'] == -1
sp_env_df.loc[mask_env.values, 'ir'] = 0.0
sp_op_df.loc[mask_op.values, 'ac_power'] = 0.0
sp_op_df.loc[mask_op.values, 'dc_power_a'] = 0.0
sp_op_df.loc[mask_op.values, 'dc_power_b'] = 0.0


#### Clean Data : Ir higher than history high limit or history low limit will be filled with the closest value

In [ ]:
mask_sp_max_ghi = sp_env_df['ir'] > max_ghi
mask_sp_min_ghi = sp_env_df['ir'] < min_ghi
mask_sp_max_temp = sp_env_df['ambient_temp2'] > max_temp
mask_sp_min_temp = sp_env_df['ambient_temp2'] < min_temp
mask_sp_max_rh = sp_env_df['ambient_rh'] > max_rh
mask_sp_min_rh = sp_env_df['ambient_rh'] < min_rh
mask_ae_max_ghi = ae_df['GHI'] > max_ghi
mask_ae_min_ghi = ae_df['GHI'] < min_ghi

sp_env_df.loc[mask_sp_max_ghi, 'ir'] = np.nan
sp_env_df.loc[mask_sp_min_ghi, 'ir'] = np.nan
sp_env_df.loc[mask_sp_max_temp, 'ambient_temp2'] = np.nan
sp_env_df.loc[mask_sp_min_temp, 'ambient_temp2'] = np.nan
sp_env_df.loc[mask_sp_max_rh, 'ambient_rh'] = np.nan
sp_env_df.loc[mask_sp_min_rh, 'ambient_rh'] = np.nan
ae_df.loc[mask_ae_max_ghi, 'GHI'] = np.nan
ae_df.loc[mask_ae_min_ghi, 'GHI'] = np.nan

# Use forward fill method (ffill) - updated to use ffill() instead of deprecated method parameter
sp_env_df['ir'] = sp_env_df['ir'].ffill()
sp_env_df['ambient_temp2'] = sp_env_df['ambient_temp2'].ffill()
sp_env_df['ambient_rh'] = sp_env_df['ambient_rh'].ffill()
ae_df['GHI'] = ae_df['GHI'].ffill()


### Load Dominion Energy (PQDIF) Data 

## Insert to MySQL Database

### Make Connection to MySQL : mysql_obj

In [ ]:
def mysql_connect():
    """Connect to MySQL database using credentials from config."""
    mysql_connect_object = mysql_lib.mySQLConnect(
        MYSQL_USERNAME, 
        MYSQL_PASSWORD, 
        MYSQL_DATABASE, 
        MYSQL_HOST
    )
    mysql_connect_object.connect()
    
    return mysql_connect_object

In [ ]:
mysql_obj = mysql_connect()
#mysql_obj.query("SELECT SunnyPortal.deviceID FROM SunnyPortal WHERE SunnyPortal.time BETWEEN '2022-07-22 23:47:00' AND '2022-07-22 23:48:00' ")
mysql_obj.query("SELECT * FROM SunnyPortal WHERE SunnyPortal.time = '2022-07-22 23:47:00' AND SunnyPortal.deviceID=29")

### For Also Energy Data

#### Adjust Data Format

In [ ]:
'''
AE Description:

    AE = {'Time','GHI','POA','ambient_temp','module_temp'}

'''
ae_df

#### Insert the dataframe Day by Day

In [ ]:
for i in range(total_days):
    current_date = start_date + timedelta(days=i)
    current_df = ae_df.iloc[ae_start_index[i]:ae_end_index[i]]
    # Insert Operation
    print(current_date)
    print(mysql_obj.insert(current_df, "AlsoEnergy"))

### For Weather Station Data

#### Calculate Weather Points

In [ ]:
severe_weather = ['Haze', 'Thunder', 'Storm', 'Heavy', 'Drizzle', 'T-Storm']  # 10 points
mild_weather = ['Cloudy', 'Rain', 'Fog', 'Smoke', 'Mist']  # 5 points

# Count weather condition occurrences
weather_condition = dict()
for i in range(ws_df.shape[0]):
    condition = ws_df['weather_condition'][i]
    if condition not in weather_condition.keys():
        weather_condition[condition] = 0
    else:
        weather_condition[condition] = weather_condition[condition] + 1

# Calculate weather scores
ws_df['weather_score'] = 0
for i in range(ws_df.shape[0]):
    points = 0
    condition_str = str(ws_df['weather_condition'][i])
    # Split by "/" to get individual weather conditions, then check each word
    weather_parts = condition_str.split("/")
    for part in weather_parts:
        part = part.strip()  # Remove whitespace
        if part in severe_weather:
            points = points + 10
        elif part in mild_weather:
            points = points + 5
    ws_df.loc[i, 'weather_score'] = points

#### Adjust Data Format

In [ ]:
'time ', 'ambient_temperature',  'relative_humidity',  'weather_condition ',' weather_score'

ws_df

#### Insert the dataframe Day by Day

In [ ]:
for i in range(total_days):
    current_date = start_date + timedelta(days=i)
    current_df = ws_df.iloc[ws_start_index[i]:ws_end_index[i]]  # Fixed: should use ws_start_index/ws_end_index
    # Insert Operation
    print(current_date)
    print(mysql_obj.insert(current_df, "WeatherStation"))

### For SunnyPortal

#### Adjust Data Format

In [ ]:
sp_df_origin = pd.concat([sp_op_df, sp_env_df],axis=1)
sp_df = pd.concat([sp_df_origin.iloc[:,:27], sp_df_origin.iloc[:,32:33], sp_df_origin.iloc[:,30:32], sp_df_origin.iloc[:,33:36], sp_df_origin.iloc[:,27:28]], axis=1)
sp_df.columns = ['time', 'ac_power', 'ac_power_l1', 'ac_power_l2', 'ac_power_l3',
       'ac_reactive_power', 'ac_reactive_power_l1', 'ac_reactive_power_l2',
       'ac_reactive_power_l3', 'ac_apparent_power', 'ac_apparent_power_l1',
       'ac_apparent_power_l2', 'ac_apparent_power_l3', 'ac_voltage_l1',
       'ac_voltage_l2', 'ac_voltage_l3', 'ac_current_l1', 'ac_current_l2',
       'ac_current_l3', 'grid_frequency', 'dc_power_a', 'dc_power_b',
       'dc_voltage_a', 'dc_voltage_b', 'dc_current_a', 'dc_current_b', 'iso', 'ir',
       'ambient_temp', 'ambient_rh', 'cap_temp', 'relay_temp', 'rh', 'deviceID']

#### Insert the Dataframe Day by Day

In [ ]:
for i in range(total_days):
    current_date = start_date + timedelta(days=i)
    current_df = sp_df.iloc[sp_start_index[i]:sp_end_index[i]]
    # Insert Operation
    print(current_date)
    print(mysql_obj.insert(current_df, "SunnyPortal"))